In [1]:
import pandas as pd 
import numpy as np 
import xarray as xr
np.warnings.filterwarnings('ignore')
import os

import os

def get_files_in_folder(folder_path):
    file_list = []
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            file_list.append(file_path)
    return file_list

# Example usage:
folder_path = '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover'  # Replace with your folder path
files = get_files_in_folder(folder_path)
print(files)


['/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2016-C3S-LC-L4-LCCS-Map-300m-P1Y-v2.1.1.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2009-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2015-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2004-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2019-C3S-LC-L4-LCCS-Map-300m-P1Y-v2.1.1.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2007-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2014-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2001-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc', '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2006-E

In [20]:
files

['/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2016-C3S-LC-L4-LCCS-Map-300m-P1Y-v2.1.1.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2009-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2015-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2004-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2019-C3S-LC-L4-LCCS-Map-300m-P1Y-v2.1.1.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2007-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2014-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover/2001-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc',
 '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcove

In [12]:
def round_to_nearest_025(x):
    return np.round(x * 4) / 4

def area_preprocessing(file_path):
    area_ds = xr.open_dataset(file_path, engine='netcdf4')
    area_ds['lat'] = xr.apply_ufunc(round_to_nearest_025, area_ds['lat'])
    area_ds['lon'] = xr.apply_ufunc(round_to_nearest_025, area_ds['lon'])
    filtered_data = area_ds.squeeze('time')
    filtered_data = filtered_data.sel(lat=slice(52, 33), lon=slice(-10, 50))
    filtered_data = filtered_data[['lat_bounds',
                                   'lon_bounds',
                                      'processed_flag',
                                      'lccs_class',
                                      'current_pixel_state',
                                      'observation_count',
                                      'change_count'
                                   ]]
    filtered_data = filtered_data.drop_duplicates(dim=['lat','lon'])
    area_df = filtered_data.to_dataframe().reset_index()
    area_df = area_df.drop(['bounds'], axis = 1)

    return area_df

In [13]:
area_preprocessing('dataset_landcover/2001-ESACCI-LC-L4-LCCS-Map-300m-v2.0.7cds.nc')

,lat,lon,lat_bounds,lon_bounds,processed_flag,lccs_class,current_pixel_state,observation_count,change_count,time
0,52.0,-10.00,52.125000,-10.125000,1.0,130,1.0,119,0,2001-01-01
1,52.0,-9.75,52.125000,-9.875000,1.0,210,2.0,156,0,2001-01-01
2,52.0,-9.50,52.125000,-9.625000,1.0,130,1.0,100,0,2001-01-01
3,52.0,-9.25,52.125000,-9.375000,1.0,130,1.0,87,0,2001-01-01
4,52.0,-9.00,52.125000,-9.125000,1.0,130,1.0,100,0,2001-01-01
...,...,...,...,...,...,...,...,...,...,...
37109,33.0,49.00,33.122222,48.877778,1.0,120,1.0,391,0,2001-01-01
37110,33.0,49.25,33.122222,49.127778,1.0,30,1.0,263,0,2001-01-01
37111,33.0,49.50,33.122222,49.377778,1.0,10,1.0,322,0,2001-01-01
37112,33.0,49.75,33.122222,49.627778,1.0,11,1.0,374,0,2001-01-01


In [31]:
import os
import pandas as pd
import xarray as xr
import numpy as np

def round_to_nearest_025(value):
    return np.round(value / 0.25) * 0.25

def area_preprocessing(file_path):
    area_ds = xr.open_dataset(file_path, engine='netcdf4')
    area_ds['lat'] = xr.apply_ufunc(round_to_nearest_025, area_ds['lat'])
    area_ds['lon'] = xr.apply_ufunc(round_to_nearest_025, area_ds['lon'])
    filtered_data = area_ds.squeeze('time')
    filtered_data = filtered_data.sel(lat=slice(52, 33), lon=slice(-10, 50))
    filtered_data = filtered_data[['lat_bounds',
                                   'lon_bounds',
                                   'processed_flag',
                                   'lccs_class',
                                   'current_pixel_state',
                                   'observation_count',
                                   'change_count'
                                  ]]
    filtered_data = filtered_data.drop_duplicates(dim=['lat', 'lon'])
    area_df = filtered_data.to_dataframe().reset_index()
    area_df = area_df.drop(['bounds'], axis=1)

    return area_df

# List of file paths
file_paths = []
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        file_paths.append(file_path)
       

# Example usage:
folder_path = '/home/ademmanuel_01/code/ademmanuel01/fireproject/dataset_landcover'  # Replace with your folder path

# Creating an empty DataFrame to store the results
combined_df = pd.DataFrame()

# Iterating over the file paths and applying the area_preprocessing function
for file_path in file_paths:
    df = area_preprocessing(file_path)
    combined_df = combined_df.append(df)

# Resetting the index of the combined DataFrame
combined_df = combined_df.reset_index(drop=True)

# Print the combined DataFrame
print(combined_df)


         lat    lon  lat_bounds  lon_bounds  processed_flag  lccs_class  \
0       52.0 -10.00   52.125000  -10.125000             1.0         130   
1       52.0  -9.75   52.125000   -9.875000             1.0         210   
2       52.0  -9.50   52.125000   -9.625000             1.0         130   
3       52.0  -9.25   52.125000   -9.375000             1.0         130   
4       52.0  -9.00   52.125000   -9.125000             1.0         130   
...      ...    ...         ...         ...             ...         ...   
742275  33.0  49.00   33.122222   48.877778             1.0         120   
742276  33.0  49.25   33.122222   49.127778             1.0          30   
742277  33.0  49.50   33.122222   49.377778             1.0          10   
742278  33.0  49.75   33.122222   49.627778             1.0          11   
742279  33.0  50.00   33.122222   49.877778             1.0          30   

        current_pixel_state  observation_count  change_count       time  
0                       1

In [35]:
combined_df['time'] = pd.to_datetime(combined_df['time'], format='%Y')

In [42]:
combined_df['time'] = pd.to_datetime(combined_df['time']).dt.year

# Sorting the DataFrame by the 'date' column in ascending order
combined_df_sorted = combined_df.sort_values('time', ascending=True)

# Resetting the index of the sorted DataFrame
combined_df_sorted = combined_df_sorted.reset_index(drop=True)

# Printing the sorted DataFrame
combined_df_sorted

,lat,lon,lat_bounds,lon_bounds,processed_flag,lccs_class,current_pixel_state,observation_count,change_count,time
0,52.00,-10.00,52.125000,-10.125000,1.0,130,1.0,119,0,2001
1,39.25,29.50,39.375000,29.375000,1.0,100,1.0,333,0,2001
2,39.25,29.75,39.375000,29.625000,1.0,100,1.0,302,1,2001
3,39.25,30.00,39.375000,29.875000,1.0,70,1.0,276,0,2001
4,39.25,30.25,39.375000,30.125000,1.0,10,1.0,332,0,2001
...,...,...,...,...,...,...,...,...,...,...
742275,39.25,29.50,39.375000,29.375000,1.0,100,1.0,333,0,2020
742276,39.25,29.25,39.375000,29.125000,1.0,70,1.0,250,0,2020
742277,39.25,29.00,39.375000,28.875000,1.0,70,1.0,271,0,2020
742278,39.25,42.00,39.375000,41.875000,1.0,10,1.0,325,0,2020


In [43]:
combined_df_sorted.to_csv('satellite_landcover_dataset.csv', index=False)